In [1]:
import pandas as pd
import numpy as np
import spacy
nlp = spacy.load("en_core_web_sm")
import json
from spacy.lemmatizer import Lemmatizer, ADJ, NOUN, VERB

lemmatizer = nlp.vocab.morphology.lemmatizer

In [2]:
import numpy as np
import operator
fileList=['GPTBaseline','DistilBertBaseline','GPTLargeBaseline']
resultDf=pd.DataFrame()
for num,fileName in enumerate(fileList):
    print(fileName)
    dfs=pd.read_excel("FinalDatasetAuto"+fileName+".xlsx",index_col=0)
    dfs=dfs[0:427]
    for index,i in enumerate(dfs['adj/noun']):
        if dfs.loc[index,'adj/noun-label']=='A' and (dfs.loc[index,'adverb']=='X' or pd.isnull(dfs.loc[index,'adverb'])):
            dfs.loc[index,'arg0']=''
            dfs.loc[index,'arg1']=''
            dfs.loc[index,'V']=''
            dfs.loc[index,'baseV']=''
            dfs.loc[index,'PP']=''
        if dfs.loc[index,'adj/noun-label']=='A' and not pd.isnull(dfs.loc[index,'adverb']):
            if dfs.loc[index,'adverb']!='X':                                                    
                dfs.loc[index,'adj/noun-label']='AB'
    dfs['arg0']=dfs['arg0'].replace(np.NaN,'')
    dfs['arg1']=dfs['arg1'].replace(np.NaN,'')
    dfs['PP']=dfs['PP'].replace(np.NaN,'')
    dfs['adverb']=dfs['adverb'].replace(np.NaN,'')
    columnList=['arg0','baseV','arg1','PP','adverb']
    for index,i in enumerate(dfs['adj/noun']):
        result=''
        for column in columnList:
            if dfs.loc[index,column]!='' and dfs.loc[index,column]!='X':
                    result=result+dfs.loc[index,column]+" "
        dfs.loc[index,'gold']=result.strip()
    for index,i in enumerate(dfs['adj/noun']):
        if index+1<len(dfs['adj/noun']) and dfs.loc[index,'adj/noun']==dfs.loc[index+1,'adj/noun'] and dfs.loc[index,'n_v']==dfs.loc[index+1,'n_v'] and dfs.loc[index,'prep']==dfs.loc[index+1,'prep'] and dfs.loc[index,'pobj']==dfs.loc[index+1,'pobj']:
            dfs.loc[index,'gold2']=dfs.loc[index+1,'gold']
        else:
            dfs.loc[index,'gold2']=dfs.loc[index,'gold']
    for index,i in enumerate(dfs['gold']):
        if not pd.isnull(i):
            doc=nlp(i)
            reducedGold=''
            for word in doc:
                if word.tag_!='DT':
                        reducedGold=reducedGold+word.lemma_+" "
            dfs.loc[index,'reducedGold']=reducedGold.lower()
            reducedGold3=''
            for para in dfs.loc[index,'gold'].split():
                doc=nlp(para)
                for word in doc:
                    if word.tag_!='DT':
                        if word.text!='a' and word.text!='an' and word.text!='the':
                            answer=lemmatizer(word.text, NOUN)
                            reducedGold3=reducedGold3+answer[0]+" "
            dfs.loc[index,'reducedGold3']=reducedGold3.lower()
    for index,i in enumerate(dfs['gold2']):
        if not pd.isnull(i):
            doc=nlp(i)
            reducedGold=''
            for word in doc:
                if word.tag_!='DT':
                        reducedGold=reducedGold+word.lemma_+" "
            dfs.loc[index,'reducedGold2']=reducedGold.lower()
            reducedGold4=''
            for para in dfs.loc[index,'gold2'].split():
                doc=nlp(para)
                for word in doc:
                    if word.tag_!='DT':
                        if word.text!='a' and word.text!='an' and word.text!='the':
                            answer=lemmatizer(word.text, NOUN)
                            reducedGold4=reducedGold4+answer[0]+" "
            dfs.loc[index,'reducedGold4']=reducedGold4.lower()
    dfs2=pd.read_excel("FinalDatasetAutoTextualEntailmentBaselineTop3RemoveArticleMetric.xlsx")
    dfs['HighestScorePattern']=dfs2['HighestScorePattern']
    for index,i in enumerate(dfs2['adj/noun']):
        if not pd.isnull(dfs.loc[index,'v']):
            scoreDict={}
            for pattern in dfs.loc[index,'HighestScorePattern'].split(","):
                scoreDict[pattern]=dfs.loc[index,pattern]
            dfs.loc[index,'HighestScorePattern']=max(scoreDict.items(), key=operator.itemgetter(1))[0][:-5]
        if dfs.loc[index,'HighestScorePattern']=='BVO':
            dfs.loc[index,'HighestScorePattern']='VOB'
        if dfs.loc[index,'HighestScorePattern']=='OBV':
            dfs.loc[index,'HighestScorePattern']='OVB'
    for index,i in enumerate(dfs['adj/noun']):
        #print(index)
        dfs.loc[index,'OriginalHighestScorePattern']=dfs.loc[index,'HighestScorePattern']
        if dfs.loc[index,'HighestScorePattern']=='MVpOPassive':
            pattern=dfs.loc[index,'HighestScorePattern']
            if "by" in dfs.loc[index,pattern].split():
                dfs.loc[index,'HighestScorePattern']='OVM'
            else:  
                dfs.loc[index,'HighestScorePattern']='MVpOPassiveVMpO'
        if dfs.loc[index,'HighestScorePattern']=='MpOVPassive':
                dfs.loc[index,'HighestScorePattern']='MpOVPassiveVMpO'
        if dfs.loc[index,'HighestScorePattern']=='OVpMPassive':
            pattern=dfs.loc[index,'HighestScorePattern']
            if "by" in dfs.loc[index,pattern].split():
                dfs.loc[index,'HighestScorePattern']='MVO'
            else:
                dfs.loc[index,'HighestScorePattern']='OVpMPassiveVOpM'
    for index,i in enumerate(dfs['adj/noun']):
        if not pd.isnull(dfs.loc[index,'v']):
            pattern=dfs.loc[index,'HighestScorePattern']
            if dfs.loc[index,'HighestScorePattern']!='V1' and dfs.loc[index,'HighestScorePattern']!='V2' and dfs.loc[index,'HighestScorePattern']!='M' and dfs.loc[index,'HighestScorePattern']!='originalNominalization':
                dfs.loc[index,'HighestScorePatternSentence']=dfs.loc[index,pattern]
                doc=nlp(dfs.loc[index,pattern])
                reducedParaphrase=''
                for word in doc:
                    if word.tag_!='DT':
                            reducedParaphrase=reducedParaphrase+word.lemma_+" "
                dfs.loc[index,'reducedParaphrase']=reducedParaphrase.lower()
                reducedParaphrase2=''
                for para in dfs.loc[index,pattern].split():
                    doc=nlp(para)
                    for word in doc:
                        if word.tag_!='DT':
                            if word.text!='a' and word.text!='an' and word.text!='the':
                                answer=lemmatizer(word.text, NOUN)
                                reducedParaphrase2=reducedParaphrase2+answer[0]+" "
                dfs.loc[index,'reducedParaphrase2']=reducedParaphrase2.lower()
            else:
                dfs.loc[index,'reducedParaphrase']=''
                dfs.loc[index,'reducedParaphrase2']=''
        else:
            dfs.loc[index,'reducedParaphrase']=''
            dfs.loc[index,'reducedParaphrase2']=''
    for index,i in enumerate(dfs['adj/noun']):
        if (dfs.loc[index,'reducedParaphrase']==dfs.loc[index,'reducedGold']) or (dfs.loc[index,'reducedParaphrase']==dfs.loc[index,'reducedGold2']):
            dfs.loc[index,'FinalLabel']='True'
        elif (dfs.loc[index,'reducedParaphrase2']==dfs.loc[index,'reducedGold']) or (dfs.loc[index,'reducedParaphrase2']==dfs.loc[index,'reducedGold2']) or (dfs.loc[index,'reducedParaphrase2']==dfs.loc[index,'reducedGold3']) or (dfs.loc[index,'reducedParaphrase2']==dfs.loc[index,'reducedGold4']):
            dfs.loc[index,'FinalLabel']='True'
        else:
            dfs.loc[index,'FinalLabel']='False'
    for index,i in enumerate(dfs['sentence']):
        if dfs.loc[index,'HighestScorePattern']=='M' or dfs.loc[index,'HighestScorePattern']=='V1' or dfs.loc[index,'HighestScorePattern']=='V2' or dfs.loc[index,'HighestScorePattern']=='originalNominalization':
            dfs.loc[index,'prediction2']='M'
        elif dfs.loc[index,'HighestScorePattern']=='VOB' or dfs.loc[index,'HighestScorePattern']=='OVB' or dfs.loc[index,'HighestScorePattern']=='OBV' or dfs.loc[index,'HighestScorePattern']=='BVO':
            dfs.loc[index,'prediction2']='MB'
        elif dfs.loc[index,'HighestScorePattern']=='originalNominalizationScore':
            dfs.loc[index,'prediction2']='M'
        else:
            dfs.loc[index,'prediction2']='NotM'
    for index,i in enumerate(dfs['adj/noun']):
        if (dfs.loc[index,'prediction2']=='M') and (dfs.loc[index,'adj/noun-label']=='A'):
            dfs.loc[index,'FinalLabel2']='True'
        elif (dfs.loc[index,'prediction2']=='MB') and (dfs.loc[index,'adj/noun-label']=='AB'):
            dfs.loc[index,'FinalLabel2']='True'
        elif (dfs.loc[index,'prediction2']=='NotM') and ((dfs.loc[index,'adj/noun-label']!='A') and (dfs.loc[index,'adj/noun-label']!='AB')):
            dfs.loc[index,'FinalLabel2']='True'
        else:
            dfs.loc[index,'FinalLabel2']='False'
    dfs=dfs.fillna('')
    for index,i in enumerate(dfs['adj/noun']):
        for columns in dfs.columns:
            dfs.loc[index,columns]=str(dfs.loc[index,columns])
    dfs=dfs.groupby(['adj/noun','n_v','prep','pobj'],sort=False)[dfs.columns[4:]].agg('//'.join).reset_index()
    dfs['final']=dfs['FinalLabel']
    dfs['final2']=dfs['FinalLabel2']
    dfs['finalM']=dfs['prediction2']
    for index,i in enumerate(dfs['adj/noun']):
        if 'True' in dfs.loc[index,'FinalLabel'].split("//"):
            dfs.loc[index,'FinalLabel']='True'
        else:
            dfs.loc[index,'FinalLabel']='False'
    for index,i in enumerate(dfs['adj/noun']):
        if 'True' in dfs.loc[index,'FinalLabel2'].split("//"):
            dfs.loc[index,'prediction2']=dfs.loc[index,'finalM'].split("//")[dfs.loc[index,'FinalLabel2'].split("//").index('True')]
            dfs.loc[index,'FinalLabel2']='True'
        else:
            dfs.loc[index,'prediction2']=dfs.loc[index,'finalM'].split("//")[dfs.loc[index,'FinalLabel2'].split("//").index('False')]
            dfs.loc[index,'FinalLabel2']='False'
    print(dfs['FinalLabel'].value_counts())
    print("Accuracy= "+str(dfs['FinalLabel'].value_counts()['True']/len(dfs['FinalLabel'])))
    print(dfs['FinalLabel2'].value_counts())
    print("Accuracy= "+str(dfs['FinalLabel2'].value_counts()['True']/len(dfs['FinalLabel2'])))
    resultDf.loc[num,'model']=fileName
    resultDf.loc[num,'True']=dfs['FinalLabel'].value_counts()['True']
    resultDf.loc[num,'False']=dfs['FinalLabel'].value_counts()['False']
    resultDf.loc[num,'Accuracy']=str(dfs['FinalLabel'].value_counts()['True']/len(dfs['FinalLabel']))
    dfs.to_excel("FinalDatasetAuto"+fileName+"Top3BasedTextualEntailmentRemoveArticleMetric.xlsx")
    print(len(dfs[(dfs['prediction2']=='NotM')& (dfs['FinalLabel2']=='True')]))
    print(len(dfs[(dfs['prediction2']=='NotM')]))
    print("PrecisionNotM= "+str(len(dfs[(dfs['prediction2']=='NotM')& (dfs['FinalLabel2']=='True')])/len(dfs[(dfs['prediction2']=='NotM')])))
    precisionNotM=len(dfs[(dfs['prediction2']=='NotM')& (dfs['FinalLabel2']=='True')])/len(dfs[(dfs['prediction2']=='NotM')])
    print(len(dfs[(dfs['prediction2']=='NotM')& (dfs['FinalLabel2']=='True')]))
    print((len(dfs[(dfs['prediction2']=='NotM')& (dfs['FinalLabel2']=='True')])+len(dfs[(dfs['prediction2']!='NotM')& (dfs['FinalLabel2']=='False')])))
    print("RecallNotM= "+str(len(dfs[(dfs['prediction2']=='NotM')& (dfs['FinalLabel2']=='True')])/(len(dfs[(dfs['prediction2']=='NotM')& (dfs['FinalLabel2']=='True')])+len(dfs[(dfs['prediction2']!='NotM')& (dfs['FinalLabel2']=='False')]))))
    recallNotM=len(dfs[(dfs['prediction2']=='NotM')& (dfs['FinalLabel2']=='True')])/(len(dfs[(dfs['prediction2']=='NotM')& (dfs['FinalLabel2']=='True')])+len(dfs[(dfs['prediction2']!='NotM')& (dfs['FinalLabel2']=='False')]))
    print(2 * (precisionNotM * recallNotM))
    print((precisionNotM + recallNotM))
    NotMF1=2 * (precisionNotM * recallNotM) / (precisionNotM + recallNotM)
    print("NotM F1= "+str(NotMF1))

GPTBaseline
False    276
True     100
Name: FinalLabel, dtype: int64
Accuracy= 0.26595744680851063
True     234
False    142
Name: FinalLabel2, dtype: int64
Accuracy= 0.6223404255319149
199
290
PrecisionNotM= 0.6862068965517242
199
250
RecallNotM= 0.796
1.092441379310345
1.482206896551724
NotM F1= 0.7370370370370372
DistilBertBaseline
False    276
True     100
Name: FinalLabel, dtype: int64
Accuracy= 0.26595744680851063
True     226
False    150
Name: FinalLabel2, dtype: int64
Accuracy= 0.601063829787234
192
281
PrecisionNotM= 0.6832740213523132
192
253
RecallNotM= 0.758893280632411
1.037064127269914
1.4421673019847243
NotM F1= 0.7191011235955055
GPTLargeBaseline
False    274
True     102
Name: FinalLabel, dtype: int64
Accuracy= 0.2712765957446808
True     232
False    144
Name: FinalLabel2, dtype: int64
Accuracy= 0.6170212765957447
198
290
PrecisionNotM= 0.6827586206896552
198
250
RecallNotM= 0.792
1.0814896551724138
1.4747586206896552
NotM F1= 0.7333333333333333


In [3]:
dfs[(dfs['prediction2']!='NotM')& (dfs['FinalLabel2']=='False')]

,adj/noun,n_v,prep,pobj,adj/noun-label,pobj-label,adj/noun.1,arg0,V,baseV,...,OriginalHighestScorePattern,HighestScorePatternSentence,reducedParaphrase,reducedParaphrase2,FinalLabel,prediction2,FinalLabel2,final,final2,finalM
1,choral,responses,by,the characters,PP,arg0,chorus,the characters,respond,respond,...,OVB,the characters respond chorally,character respond chorally,character respond chorally,False,MB,False,False,False,MB
6,racial,separation,of,the population,PP,arg1,race,,separate,separate,...,OVB,the population separates racially,population separate racially,population separate racially,False,MB,False,False,False,MB
8,diplomatic,intercession,of,the United States,PP,arg0,diplomacy,the United States,intercede,intercede,...,OVB,the United States intercede diplomatically,united states intercede diplomatically,united state intercede diplomatically,False,MB,False,False,False,MB
10,physical,description,of,the Buddha,arg1,arg1,physique,,describe,describe,...,OVB,the Buddha describes physically,buddha describe physically,buddha describe physically,False,MB,False,False,False,MB
14,racial,classification,of,humanity,PP,arg1,race,,classify,classify,...,OVB,humanity classifies racially,humanity classifie racially,humanity classifie racially,False,MB,False,False,False,MB
16,climatic,effects,of,air pollutants,arg1,arg0,climate,air pollutants,affect,affect,...,OVB,air pollutants cause climatically,air pollutant cause climatically,air pollutant cause climatically,False,MB,False,False,False,MB
29,democratic,development,of,the United States,arg1,PP,democracy,,develop,develop,...,OVB,the United States develop democratically,united states develop democratically,united state develop democratically,False,MB,False,False,False,MB
35,technological,effect,on,media development,arg0,arg1,technology,technology,affects,affect,...,OVB,media development causes technologically,medium development cause technologically,medium development cause technologically,False,MB,False,False,False,MB
40,mechanical,maintenance,of,aircraft,arg1,arg1,mechanics,,maintain,maintain,...,VOB,maintain aircraft mechanically,maintain aircraft mechanically,maintain aircraft mechanically,False,MB,False,False,False,MB
64,scientific,contributions,of,Muslim civilizations,PP,arg0,science,Muslim civilizations,contribute,contribute,...,OVB,Muslim civilizations contribute scientifically,muslim civilization contribute scientifically,muslim civilization contribute scientifically,False,MB,False,False,False,MB
